## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups:

....

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:

 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
 - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting
 - Addition of alternate dataloader with Fixed-Length Truncation
 - Addition of performance analysis for each category

....

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
                 analyze_test_set_performance, eval_test_set_by_category, eval_test_set_pure, display_category_examples
from my_distilbert import QADataset

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Load data_2 (Traditional split)

Note: QADataset loader is the Fixed-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create training dataset using only SQuAD data, DataLoader with batch size of 8
dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None, tokenizer=tokenizer)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8)
print(f"Approximate Training Dataset Size: {len(dataset_2)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Approximate Training Dataset Size: 113000
Approximate Validation Dataset Size: 14000
Approximate Test Dataset Size: 14000


# Model_22
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split; Alter training parameters with 3 epochs, dropout rate of 0.18, AdamW optimzer and max tokens at 384 rather than 512

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_22 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_22 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_22 = model_22.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_22 = SimpleQuestionDistilBERT(mod_22)
model_22.set_dropout_rate(0.18)
model_22.to(device)

# Verify the dropout rates for each layer
for name, module in model_22.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.18
distilbert.transformer.layer.0.attention.dropout: 0.18
distilbert.transformer.layer.0.ffn.dropout: 0.18
distilbert.transformer.layer.1.attention.dropout: 0.18
distilbert.transformer.layer.1.ffn.dropout: 0.18
distilbert.transformer.layer.2.attention.dropout: 0.18
distilbert.transformer.layer.2.ffn.dropout: 0.18
distilbert.transformer.layer.3.attention.dropout: 0.18
distilbert.transformer.layer.3.ffn.dropout: 0.18
distilbert.transformer.layer.4.attention.dropout: 0.18
distilbert.transformer.layer.4.ffn.dropout: 0.18
distilbert.transformer.layer.5.attention.dropout: 0.18
distilbert.transformer.layer.5.ffn.dropout: 0.18
dropout: 0.18


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_22.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = AdamW(model_22.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 3 epochs
# Validation data used to evaluate performance during training

epochs = 3

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_22.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_22(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_22.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_22(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.4773100832669082


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.153898595181959


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.1435356094618994


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.7705330109295592


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.2068810580821177


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_22.state_dict(), "simple_distilbert_qa_data_22.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_22 = SimpleQuestionDistilBERT(mod_22)

# Load previously saved model parameters from disk
model_22.load_state_dict(torch.load("simple_distilbert_qa_data_22.model"))

<ipython-input-10-49cfa6766c1f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_22.load_state_dict(torch.load("simple_distilbert_qa_data_22.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_22, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:02<00:00, 56.06it/s]


Mean EM:  0.6357142857142857
Mean F-1:  0.703671052061327


In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set_pure(model_22, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [00:50<00:00, 69.11it/s]

Mean Pure EM:  0.6058571428571429
Mean Pure F-1:  0.6937455235123801


In [ ]:
# Then add category evaluation
print("\nAnalyzing performance by question category:")
eval_test_set_by_category(model_22, tokenizer, test_loader_2, device)


Analyzing performance by question category:


100%|██████████| 3500/3500 [00:53<00:00, 66.02it/s]

+---------------+-------+------------+---------+---------+
| Question Type | Count | % of Total | Mean EM | Mean F1 |
+---------------+-------+------------+---------+---------+
|      what     |  8084 |   57.7%    |  0.627  |  0.697  |
|      when     |  946  |    6.8%    |  0.693  |  0.747  |
|      how      |  1400 |   10.0%    |  0.595  |  0.696  |
|      who      |  1453 |   10.4%    |  0.710  |  0.755  |
|      why      |  175  |    1.2%    |  0.434  |  0.558  |
|     other     |  278  |    2.0%    |  0.608  |  0.674  |
|     where     |  619  |    4.4%    |  0.624  |  0.709  |
|     which     |  1045 |    7.5%    |  0.619  |  0.694  |
+---------------+-------+------------+---------+---------+


{'what': {'count': 8084,
  'percentage': 57.74285714285714,
  'mean_em': 0.6270410687778327,
  'mean_f1': 0.6971736116224101},
 'when': {'count': 946,
  'percentage': 6.757142857142857,
  'mean_em': 0.693446088794926,
  'mean_f1': 0.7473496205976381},
 'how': {'count': 1400,
  'percentage': 10.0,
  'mean_em': 0.595,
  'mean_f1': 0.6955382691636628},
 'who': {'count': 1453,
  'percentage': 10.378571428571428,
  'mean_em': 0.7102546455609084,
  'mean_f1': 0.7545276571381822},
 'why': {'count': 175,
  'percentage': 1.25,
  'mean_em': 0.4342857142857143,
  'mean_f1': 0.5584712990369859},
 'other': {'count': 278,
  'percentage': 1.9857142857142858,
  'mean_em': 0.6079136690647482,
  'mean_f1': 0.6740189275378128},
 'where': {'count': 619,
  'percentage': 4.421428571428572,
  'mean_em': 0.6235864297253635,
  'mean_f1': 0.7090885290700838},
 'which': {'count': 1045,
  'percentage': 7.464285714285715,
  'mean_em': 0.6191387559808612,
  'mean_f1': 0.6935645164472416}}

In [ ]:
# Get examples by category
examples = display_category_examples(model_22, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [00:53<00:00, 65.62it/s]


==================== WHAT Questions ====================

✅ Successful Examples (EM=1):

1. Question: in what year was plymouth recognized as a town? the first record of the existence of a settlement at plymouth was in the domesday book in 1086 as sudtone, saxon for south farm, located at the present day barbican. from saxon times, it was in the hundred of roborough. in 1254 it gained status as a town and in 1439, became the first town in england to be granted a charter by parliament. between 1439 and 1934, plymouth had a mayor. in 1914 the county boroughs of plymouth and devonport, and the urban district of east stonehouse merged to form a single county borough of plymouth. collectively they were referred to as " the three towns ".
   Predicted: 1254
   Truth: 1254

2. Question: what is the name of the bungie inc. founder who is also a university graduate? in the arts and entertainment, minimalist composer philip glass, dancer, choreographer and leader in the field of dance anthropol